# Output Parsers

In [4]:
from langchain_ollama import ChatOllama

base_url = "http://localhost:11434"
model = 'llama3.2:1b'
# Initialize the Llama model 
llm = ChatOllama(
    base_url=base_url,
    model = model,
    temperature = 0.8,
    num_predict = 256
)

# Chain Processing without / with the output parser

In [5]:
from langchain_core.output_parsers import StrOutputParser
# Without parser
print("--- Without Parser ---")
response = llm.invoke("What's the weather in San Francisco?")
print("Type:", type(response))
print("Content:", response.content)


# With StrOutputParser
print("\n--- With StrOutputParser ---")
chain = llm | StrOutputParser()
response = chain.invoke("What's the weather in San Francisco?")
print("Type:", type(response))
print("Content:", response)

--- Without Parser ---
Type: <class 'langchain_core.messages.ai.AIMessage'>
Content: As I'm a large language model, I don't have real-time access to current weather conditions. However, I can suggest some ways for you to find out the current weather in San Francisco.

You can check the weather forecast for San Francisco by visiting a weather website or app, such as:

* National Weather Service (weather.gov)
* AccuWeather (accuweather.com)
* Weather Underground (wunderground.com)

These websites will provide you with the current weather conditions, including temperature, humidity, wind speed, and precipitation forecasts, for San Francisco.

If you're looking for a specific date or period of time, I can try to help you find a reliable source of information.

--- With StrOutputParser ---
Type: <class 'str'>
Content: In my knowledge cutoff of March 2023, I can provide you with general information about the typical weather patterns in San Francisco. However, please note that this informatio

# StructuredOutputParser

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI

# Define response schemas for payment processing information
response_schemas = [
    ResponseSchema(
        name="company_name", 
        description="Name of the payment processing company"
    ),
    ResponseSchema(
        name="supported_countries", 
        description="List of countries where the payment processor operates"
    ),
    ResponseSchema(
        name="supported_card_types", 
        description="Types of credit/debit cards supported"
    ),
    ResponseSchema(
        name="key_features", 
        description="Key features and unique selling points of the payment processor"
    ),
    ResponseSchema(
        name="transaction_fees", 
        description="Overview of transaction fees or pricing model"
    )
]

# Create a structured output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get format instructions for the output
format_instructions = output_parser.get_format_instructions()

# Create a prompt template specific to payment processing companies
prompt = PromptTemplate(
    template="""You are an expert in payment processing technologies. 
Provide a comprehensive overview of the payment processing company based on the query.

{format_instructions}

Query: {question}

Provide detailed, accurate information about the payment processor, focusing on its capabilities, market presence, and unique characteristics.""",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

# Initialize the ChatOpenAI model
# model = ChatOpenAI(temperature=0.3, model="gpt-4-turbo")

# Create the processing chain
chain = prompt | llm | output_parser

# Example queries about payment processors
print("Stripe Overview:")
stripe_info = chain.invoke({"question": "Provide details about Stripe payment processing"})
print(stripe_info)




Stripe Overview:
{'company_name': 'Stripe', 'supported_countries': 'Global (over 180 countries)', 'supported_card_types': 'Visa, Mastercard, Amex, Discover, JCB', 'key_features': 'Real-time transactions, secure and scalable payment processing, PCI-DSS compliant, multi-currency support, real-time API for custom integrations', 'transaction_fees': 'Competitive pricing model, transaction fees range from $0.000004 to $3.99 per transaction, no monthly or yearly subscription fees'}


In [7]:
base_url = "http://localhost:11434"
model = 'llama3.2:3b'
# Initialize the Llama model 
llm = ChatOllama(
    base_url=base_url,
    model = model,
    temperature = 0.8,
    num_predict = 256
)
# Pydantic
from typing import List
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# Define a Pydantic model for payment processing information
class PaymentProcessorInfo(BaseModel):
    company_name: str = Field(description="Name of the payment processing company")
    supported_countries: List[str] = Field(description="List of countries where the payment processor operates")
    supported_card_types: List[str] = Field(description="Types of credit/debit cards supported")
    key_features: List[str] = Field(description="Key features and unique selling points of the payment processor")
    transaction_fees: str = Field(description="Overview of transaction fees or pricing model")

# Create a Pydantic output parser
output_parser = PydanticOutputParser(pydantic_object=PaymentProcessorInfo)

# Create a prompt template specific to payment processing companies
prompt = PromptTemplate(
    template="""You are an expert in payment processing technologies. 
Provide a comprehensive overview of the payment processing company based on the query.

{format_instructions}

Query: {question}

Provide  detailed, accurate information about the payment processor,that complies with the schema,  focusing on its capabilities, market presence, and unique characteristics.""",
    input_variables=["question"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

#  # Initialize the ChatOpenAI model
# model = ChatOpenAI(temperature=0.3) 

# Create the processing chain
chain = prompt | llm | output_parser

# Example queries about payment processors
try:
    print("Stripe Overview:")
    stripe_info = chain.invoke({"question": "Provide details about Stripe payment processing"})
    print(stripe_info)

    
except Exception as e:
    print(f"An error occurred: {e}")


Stripe Overview:
An error occurred: Failed to parse PaymentProcessorInfo from completion {"company_name": "Stripe", "description": "A financial services company that provides online payment processing solutions.", "supported_countries": ["AF", "AX", "AW", "AD", "AE", "AL", "AM", "AO", "AZ", "BA", "BH", "BD", "BB", "BY", "BZ", "BM", "BF", "BG", "BI", "KH", "CM", "CV", "CI", "CF", "CG", "CD", "CN", "CK", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EG", "SV", "GQ", "ER", "EE", "ET", "FI", "FJ", "FR", "GF", "PF", "VG", "GU"]}. Got: 3 validation errors for PaymentProcessorInfo
supported_card_types
  Field required [type=missing, input_value={'company_name': 'Stripe'...'GF', 'PF', 'VG', 'GU']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
key_features
  Field required [type=missing, input_value={'company_name': 'Stripe'...'GF', 'PF', 'VG', 'GU']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2